In [262]:
import pandas as pd
import matplotlib.pyplot as plt
from typing import Optional 
import numpy as np
INITIAL_BALANCE = 5000

In [263]:
df = pd.read_excel(r"Files/the5ers_bots.xlsx",sheet_name="All")
df = pd.read_excel(r"Files/TTP_bots.xlsx",sheet_name="All")

#Rellenando na con ceros 
df.fillna(0,inplace=True)


#Colocando comment en operaciones manuales
df.loc[df["Comment"]==np.nan,"Comment"] = "Manual"

#Resetando indice
df.reset_index(inplace=True,drop=True)

#Rename a dos columnas que se llamaban iguales
df.rename(columns={"Price":"Price Entry", "Price.1":"Price Exit"},inplace=True)
df.rename(columns={"Time":"Time Entry","Time.1":"Time Exit"},inplace=True)

#Nueva columna con el balance 
df["Balance"] = INITIAL_BALANCE + (df["Profit"]+df["Commission"]+df["Swap"]).cumsum() 


# Funciones

In [264]:
def get_bots_by_comment():
    return df["Comment"].unique()

def handle_errors_not_string_and_not_in_df(x):
     #Verificicar que sea none para no entrar en el if y que sea una string 
    if x is not None:
        if  not isinstance(x,str) :
            raise TypeError(f"{x} no es una string")
    
        #Conseguir los bots por comentario del df y verificar si bot se encuentra
        if x not in get_bots_by_comment():
            raise KeyError(f"{x} no está en el dataframe")    

In [265]:
def profit_factor_v2(data,by_bot=True):
    
    #Profit factor por bot
    if by_bot:
        x = data.groupby("Comment")        

        return x.apply(profit_factor_v2,by_bot=False,include_groups=False) #type: ignore


    total_profit = data[data["Profit"]>=0]["Profit"].sum()
    total_loss = data[data["Profit"]< 0]["Profit"].sum()

    #Division por 0
    if total_loss==0:
        return float('inf')
    

    return round(abs(total_profit/total_loss),2)
    

In [266]:
def win_rate(data:pd.DataFrame,by_bot = False,BE = True):
    """
    Calcula los ratios para los bots o para el total de la cuenta
    Se puede desactivar el calculo con BE y hacerlo normal

    Nota: Se considera BE cuando el profit fue menor que el 0.15% del balance actual
    """

    #data es un dataframe?
    if not isinstance(data,pd.DataFrame):
        raise TypeError("data is not a Dataframe")
    
    #Profit y balance son columnas necesarias para el calculo, están?
    if "Profit" not in data.columns or "Balance" not in data.columns:
        raise KeyError("Columns 'Profit' or 'Balance' is not in your dataframe")

  

    #Calculo
    total = data["Profit"].count()

    percent = lambda x: round(x*100/total,2)  

    win = ((data["Profit"] > 0) & (data["Profit"] > data["Balance"]*0.0015)).sum() 
    win_percent = percent(win)

    losses = (data["Profit"] < 0 ).sum()
    losses_percent = percent(losses)
    
    #Diccionario base
    dicc =  {"Win":[f"{win_percent}%",win],
            "Loss":[f"{losses_percent}%",losses]}

    #Queres los breakeven?
    if BE:
        breakeven =  ((data["Profit"] < data["Balance"]*0.0015) & (data["Profit"] > 0)).sum()
        breakeven_percent = percent(breakeven)

        dicc["BE"] = [f"{breakeven_percent}%",breakeven]
    
    #Añado el total
    
    dicc["Total"] = ["100%", total]


    #Para cada bot
    if by_bot:
        x = data.groupby("Comment")[["Profit","Balance"]]
        return x.apply(win_rate,BE=BE)
    

    return pd.DataFrame(dicc,index=["Percent","n"])

In [267]:
def point_value(data_for_point_value:pd.DataFrame):

    temp_df = data_for_point_value.copy()
    
    #*Calculo valor del punto =        |Profit|
    #*                          ----------------------------
    #*                             |P.Entry-P.Exit|*Volume 
    
    temp_df["point_value"] = temp_df["Profit"].abs()/((temp_df["Price Entry"]-temp_df["Price Exit"]).abs()*temp_df["Volume"])
    
    df_return = dict(temp_df.groupby("Symbol")["point_value"].median().round(2))
   
    return df_return

In [268]:
def add_R(data_R:pd.DataFrame):
    if "R-multiple" in data_R.columns:
        return None
    
    #Se sacan  las operaciones sin stops iniciales
    data_R.drop(data_R[data_R["S / L"] ==0].index,inplace=True)
    data_R.reset_index(drop=True,inplace=True)

    
    for i in point_value(data_R):
        
        #* Calculo R(usd): |P.Entry-SL|* Volumen* point_value

        #Operaciones sin stop inicial deben ser tratadas con otro criterio 
     
        func = abs(data_R["Price Entry"] - data_R.loc[data_R["S / L"]>0,"S / L"])*data_R["Volume"]*point_value(data_R)[i]
        
        data_R.loc[data_R["Symbol"]==i,"R(usd)"] = func.round(2)
    
    data_R["R-multiple"] = (data_R["Profit"]/data_R["R(usd)"]).round(2)

        
    return data_R 

In [269]:
def balance_in_R(data:pd.DataFrame,RISK:float = 0.01):
    
    # * Formula: Balance_n = Balance_0 * PRODUCTORIO(1 + R_n* RISK) 
    
    #Limpio "R(usd)" que no me interesa
    if "R(usd)" in data.columns:
        data = data.drop(columns="R(usd)")

    #Interior del Productorio
    data["growth"] = 1+ data["R-multiple"]*RISK
    
    #Toda la Formula
    data["Balance_with_R"] = (INITIAL_BALANCE *data["growth"].cumprod()).round(2)
    
    
    return data[data.columns[-6:]]

In [270]:
def max_drawdown_in_R(data:pd.DataFrame):
    if "R" not in data.columns: 
        add_R(data)
        print("Columna R añadida")
    
    data["R"]= data["R"].fillna(0)

    balance_n = data["R"].cumsum() 
    peak = balance_n.cummax()
    dd_in_R = balance_n - peak
    return dd_in_R.min() 



# Cosas Varias

In [ ]:
# print(df.Comment.unique())
normdf = df.drop(columns=["T / P","Position","Time Exit","Type","Balance"])

add_R(normdf)

xaum5 = df[df["Comment"]=="PORTFOLIO_1_2_3"].copy()


add_R(xaum5)
profit_factor_v2(xaum5)
xaum5["Profit"].mean()






np.float64(0.33000000000000007)

In [272]:

profit = profit_factor_v2(df).to_frame("Profit Factor") #type: ignore

df_win = win_rate(df,by_bot=True)

df_win_flat = df_win.unstack(level=-1)
df_win_flat.columns = [f"{col[0]}_{col[1]}" for col in df_win_flat.columns]



df_final = pd.merge(profit, df_win_flat, left_index=True, right_index=True, how="outer")

# df_final["Profit"] = df.groupby("Comment")["Profit-Commi-Swap"].sum()
# df_final.sort_values(by="Profit",inplace=True)

In [273]:
from logic import BotAnalyzer 